In [ ]:
!pip install git+https://github.com/Ritvik19/pyradox.git

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import sin, cos, pi
import cv2, os
from tqdm.auto import tqdm

from tensorflow.keras import layers, callbacks, utils, applications, optimizers, Input
from tensorflow.keras.models import Sequential, Model, load_model
from pyradox import convnets

In [ ]:
class config:
    NUM_EPOCHS = 100
    BATCH_SIZE = 64

# Loading Data

In [ ]:
print("Contents of input/facial-keypoints-detection directory: ")
!ls ../input/facial-keypoints-detection/

print("\nExtracting .zip dataset files to working directory ...")
!unzip -u ../input/facial-keypoints-detection/test.zip
!unzip -u ../input/facial-keypoints-detection/training.zip

print("\nCurrent working directory:")
!pwd
print("\nContents of working directory:")
!ls

In [ ]:
%%time

train_file = 'training.csv'
test_file = 'test.csv'
idlookup_file = '../input/facial-keypoints-detection/IdLookupTable.csv'
train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)
idlookup_data = pd.read_csv(idlookup_file)

In [ ]:
def plot_sample(image, keypoint, axis, title):
    image = image.reshape(96,96)
    axis.imshow(image, cmap='gray')
    axis.scatter(keypoint[0::2], keypoint[1::2], marker='x', s=20)
    plt.title(title)

## Exploring Data

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
idlookup_data.head()

In [ ]:
print("Length of train data:", len(train_data))

**Find columns having Null values and their counts**

In [ ]:
train_data.isnull().sum()

#### We can observe that approx. 68% of data is missing for several keypoints

In [ ]:
clean_train_data = train_data.dropna()
print("clean_train_data shape:", np.shape(clean_train_data))

unclean_train_data = train_data.fillna(method = 'ffill')
print("unclean_train_data shape:", np.shape(unclean_train_data))

In [ ]:
%%time

def load_images(image_data):
    images = []
    for idx, sample in image_data.iterrows():
        image = np.array(sample['Image'].split(' '), dtype=int)
        image = np.reshape(image, (96,96,1))
        images.append(image)
    images = np.array(images)/255.
    return images

def load_keypoints(keypoint_data):
    keypoint_data = keypoint_data.drop('Image',axis = 1)
    keypoint_features = []
    for idx, sample_keypoints in keypoint_data.iterrows():
        keypoint_features.append(sample_keypoints)
    keypoint_features = np.array(keypoint_features, dtype = 'float')
    return keypoint_features

clean_train_images = load_images(clean_train_data)
print("Shape of clean_train_images:", np.shape(clean_train_images))
clean_train_keypoints = load_keypoints(clean_train_data)
print("Shape of clean_train_keypoints:", np.shape(clean_train_keypoints))
test_images = load_images(test_data)
print("Shape of test_images:", np.shape(test_images))

train_images = clean_train_images
train_keypoints = clean_train_keypoints
fig, axis = plt.subplots()
plot_sample(clean_train_images[19], clean_train_keypoints[19], axis, "Sample image & keypoints")

unclean_train_images = load_images(unclean_train_data)
print("Shape of unclean_train_images:", np.shape(unclean_train_images))
unclean_train_keypoints = load_keypoints(unclean_train_data)
print("Shape of unclean_train_keypoints:", np.shape(unclean_train_keypoints))
train_images = np.concatenate((train_images, unclean_train_images))
train_keypoints = np.concatenate((train_keypoints, unclean_train_keypoints))

## Visualize Train images & corresponding Keypoints

In [ ]:
print("\nClean Train Data: ")
fig = plt.figure(figsize=(20,8))
for i in range(10):
    axis = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])
    plot_sample(clean_train_images[i], clean_train_keypoints[i], axis, "")
plt.show()

print("Unclean Train Data: ")
fig = plt.figure(figsize=(20,8))
for i in range(10):
    axis = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])
    plot_sample(unclean_train_images[i], unclean_train_keypoints[i], axis, "")
plt.show()

# Model

In [ ]:
inputs = Input(shape=(96, 96, 1))
x = layers.Convolution2D(3, (1, 1), padding='same')(inputs)
x = layers.LeakyReLU(alpha = 0.1)(x)
x = convnets.MobileNetV3(config='small')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(30)(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

In [ ]:
utils.plot_model(model, show_shapes=True, expand_nested=True)

## Training the model

In [ ]:
es = callbacks.EarlyStopping(
    monitor='loss', patience=30, verbose=1, mode='min', baseline=None, restore_best_weights=True
)

rlp = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, min_lr=1e-15, mode='min', verbose=1
)

model.compile(
    optimizer='adam', loss='mean_squared_error', metrics=['mae', 'acc']
)

history = model.fit(
    train_images, train_keypoints, epochs=int(1.5*config.NUM_EPOCHS), batch_size=config.BATCH_SIZE, 
    validation_split=0.05, callbacks=[es, rlp]
)

In [ ]:
sns.set_style('darkgrid')

fig, ax = plt.subplots(3, 1, figsize=(20, 10))
df = pd.DataFrame(history.history)
df[['mae', 'val_mae']].plot(ax=ax[0])
df[['loss', 'val_loss']].plot(ax=ax[1])
df[['acc', 'val_acc']].plot(ax=ax[2])
ax[0].set_title('Model MAE', fontsize=12)
ax[1].set_title('Model Loss', fontsize=12)
ax[2].set_title('Model Acc', fontsize=12)
fig.suptitle('Model Metrics', fontsize=18);

### Fit the model on full dataset

In [ ]:
%%time

es = callbacks.EarlyStopping(
    monitor='loss', patience=30, verbose=1, mode='min', baseline=None, restore_best_weights=True
)

rlp = callbacks.ReduceLROnPlateau(
    monitor='loss', factor=0.5, patience=5, min_lr=1e-15, mode='min', verbose=1
)


model.compile(
    optimizer=optimizers.Adam(learning_rate=history.history['lr'][-1]), loss='mean_squared_error', metrics=['mae', 'acc']
)

history = model.fit(
    train_images, train_keypoints, epochs=int(3.5*config.NUM_EPOCHS), batch_size=config.BATCH_SIZE, callbacks=[es, rlp]
)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(20, 10))
df = pd.DataFrame(history.history)
df[['mae']].plot(ax=ax[0])
df[['loss']].plot(ax=ax[1])
df[['acc']].plot(ax=ax[2])
ax[0].set_title('Model MAE', fontsize=12)
ax[1].set_title('Model Loss', fontsize=12)
ax[2].set_title('Model Acc', fontsize=12)
fig.suptitle('Model Metrics', fontsize=18);

## Predicting on Test Set

In [ ]:
%%time
 
test_preds = model.predict(test_images)

## Visualizing Test Predictions

In [ ]:
fig = plt.figure(figsize=(20,16))
for i in range(20):
    axis = fig.add_subplot(4, 5, i+1, xticks=[], yticks=[])
    plot_sample(test_images[i], test_preds[i], axis, "")

## Generating Submission File

In [ ]:
feature_names = list(idlookup_data['FeatureName'])
image_ids = list(idlookup_data['ImageId']-1)
row_ids = list(idlookup_data['RowId'])

feature_list = []
for feature in feature_names:
    feature_list.append(feature_names.index(feature))
    
predictions = []
for x,y in zip(image_ids, feature_list):
    predictions.append(test_preds[x][y])
    
row_ids = pd.Series(row_ids, name = 'RowId')
locations = pd.Series(predictions, name = 'Location')
locations = locations.clip(0.0,96.0)
submission_result = pd.concat([row_ids,locations],axis = 1)
submission_result.to_csv('submission.csv',index = False)